# Gapfilling with Universal Model

In [1]:
import cobra
from cobra.core import Metabolite, Reaction
from cobra.io import read_sbml_model, load_json_model
from cobra.flux_analysis import gapfill
import os, re
import pandas as pd

print(os.getcwd())

ext_dir = '/../../../data/external'
phenomics = '/../../phenomics'
sco_dir = '/../1.gapfill_w_sco'

# Import model
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM.xml")
model = read_sbml_model(f"{os.getcwd()}/{sco_dir}/Salb-GEM-Sco-gapfill.xml")

# Mapping of the universal reaction model
universal = load_json_model(f"{os.getcwd()}/{ext_dir}/universal_model_cobrapy.json")

/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/strep/Code/streptsd/notebooks/gapfilling/2.gapfill_w_universal


'' is not a valid SBML 'SId'.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-13


# 1. Get all untestable and false negative compounds 

In [2]:
# Import agreed biolog vs model data
agreed_bio_data = pd.read_csv(f"{os.getcwd()}{sco_dir}/agreed_bio_data_Salb_Sco.csv")
agreed_bio_data['model_simulation_0.05'] = agreed_bio_data['model_simulation_0.05'].astype('boolean')

# # Mapping between bigg and metax names
# bigg_mnmx = pd.read_json(f"{os.getcwd()}/{ext_dir}/metanetx.json")

# # Mapping files for bigg compound to all other names
# name_map = pd.read_csv(f"{os.getcwd()}/{ext_dir}/biggmodels_metabolites.txt",sep='\t')
# name_map = name_map.drop(['bigg_id', 'model_list', 'old_bigg_ids'], axis=1)

agreed_bio_data

,index,activity,chemical,category,moa,bigg,exchange,model_simulation_0.05,model_simulation_0.1,model_simulation_0.2
0,PM01_A01,False,Negative Control,carbon,"C-Source, negative control",#,#,<NA>,NaN,NaN
1,PM01_A02,True,L-Arabinose,carbon,"C-Source, carbohydrate",arab__L,EX_arab__L_e,True,True,True
2,PM01_A03,True,N-Acetyl-D-Glucosamine,carbon,"C-Source, carbohydrate",acgam,EX_acgam_e,<NA>,NaN,NaN
3,PM01_A04,False,D-Saccharic acid,carbon,"C-Source, carboxylic acid",glcr,EX_glcr_e,<NA>,NaN,NaN
4,PM01_A05,True,Succinic acid,carbon,"C-Source, carboxylic acid",succ,EX_succ_e,True,True,True
...,...,...,...,...,...,...,...,...,...,...
379,PM04_H08,False,p-Aminobenzene Sulfonic acid,phosphate & sulphur,"S-Source, organic",4abz,EX_4abz_e,<NA>,NaN,NaN
380,PM04_H09,False,Butane Sulfonic acid,phosphate & sulphur,"S-Source, organic",butso3,EX_butso3_e,<NA>,NaN,NaN
381,PM04_H10,False,2-Hydroxyethane Sulfonic acid,phosphate & sulphur,"S-Source, organic",isetac,EX_isetac_e,<NA>,NaN,NaN
382,PM04_H11,False,Methane Sulfonic acid,phosphate & sulphur,"S-Source, organic",mso3,EX_mso3_e,<NA>,NaN,NaN


In [5]:
print(f"\nFalse negative (actual true, predicted false):\n")
agreed_bio_data_nan_drop = agreed_bio_data[agreed_bio_data['model_simulation_0.05'].notna()]
false_negatives = agreed_bio_data_nan_drop[
    agreed_bio_data_nan_drop.xs("activity", axis=1)
    & ~(agreed_bio_data_nan_drop.xs("model_simulation_0.05", axis=1))
    ][["bigg","chemical", "moa", "exchange", "model_simulation_0.05"]]

print(false_negatives)

false_negatives.to_csv(os.getcwd() + "/" + 'Salb_false_negative.csv', index=True)


False negative (actual true, predicted false):

        bigg                 chemical                   moa      exchange  \
179   met__L             L-Methionine  C-Source, amino acid   EX_met__L_e   
225  citr__L             L-Citrulline  N-Source, amino acid  EX_citr__L_e   
226   hom__L             L-Homoserine  N-Source, amino acid   EX_hom__L_e   
259     thym                  Thymine       N-Source, other     EX_thym_e   
260    thymd                Thymidine       N-Source, other    EX_thymd_e   
261      ura                   Uracil       N-Source, other      EX_ura_e   
262      uri                  Uridine       N-Source, other      EX_uri_e   
302   glyc3p  DL-a-Glycerol Phosphate     P-Source, organic   EX_glyc3p_e   

     model_simulation_0.05  
179                  False  
225                  False  
226                  False  
259                  False  
260                  False  
261                  False  
262                  False  
302                  Fals

# 2. Try Gapfilling Function on false negatives

In [6]:
def basic_gapfill(bigg, model, reference, objective='growth', iter=1):
    """
    gapfill a model using reference model and cplex solver.
    
    Parameters:
    -----------
    bigg: String
    model: Model
    reference: Model
    """
    print(f"\nGapfilling {bigg} with: ")
    with reference:
        reference.solver = 'cplex'
        model.solver = 'cplex'
        model.objective = objective
        solution = gapfill(model, reference, demand_reactions=False, iterations=iter)
        print(solution)
    return solution 

def gapfill_medium(model, reference, bigg, type, exchange):
    """
    This function does gapfilling based on a chemical in the certain biolog medium and add an exchange reaction if needed"
    
    Parameters:
    -----------
    bigg (str) 
    category (str)
    exchange (str)
    """

    # when model has the exchange reaction, no need to add a reaction before gapfilling.  
    try:
        with model:
            medium = model.medium
            medium[exchange] = 0.8
            if type.startswith("C"):
                medium["EX_glc__D_e"] = 0

            elif type.startswith("N"):
                medium["EX_nh4_e"] = 0

            elif type.startswith("P"):
                medium["EX_pi_e"] = 0
            
            elif type.startswith("S"):
                medium["EX_so4_e"] = 0

            model.medium = medium
            model.reactions.EX_co2_e.bounds= (0, 1000)

            return basic_gapfill(bigg, model, reference, 'growth', 4)
    except:
        print(f"Gapfilling of the compound {bigg} - {type} - {exchange} is failed\n")
        return None


In [8]:
_model = model.copy()
with open('gapfill_reactions.txt', 'w') as file:
    for ind in false_negatives.index:
        file.write(f"Gapfilling {false_negatives['bigg'][ind]}, {false_negatives['moa'][ind]} with:\n")
        solution = gapfill_medium(_model, universal, false_negatives['bigg'][ind], false_negatives['moa'][ind], false_negatives['exchange'][ind])
        if solution:
            unique_solution = set([reaction for reaction_set in solution for reaction in reaction_set])
            reactions_str = ', '.join(str(reaction.id) for reaction in unique_solution)
            print(reactions_str)
            file.write(reactions_str + '\n\n')
        else:
            file.write('\n\n')

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpvizkxmtf.lp
Reading time = 0.01 seconds
: 1886 rows, 4524 columns, 19812 nonzeros

Gapfilling met__L with: 
[[<Reaction SLGTH at 0x7fc29757bdf0>], [<Reaction GLYFEM at 0x7fc2a7037490>], [<Reaction SUCptspp_1 at 0x7fc2604925e0>], [<Reaction FACT at 0x7fc2819f2dc0>]]
GLYFEM, FACT, SUCptspp_1, SLGTH

Gapfilling citr__L with: 
[[<Reaction SLGTH at 0x7fc25055aac0>], [<Reaction OCBT2i at 0x7fc2215c9400>], [<Reaction SDPDS_2 at 0x7fc2484b00d0>], [<Reaction ARGDC at 0x7fc269a08400>]]
OCBT2i, ARGDC, SLGTH, SDPDS_2

Gapfilling hom__L with: 
[[<Reaction SLGTH at 0x7fc1e3f34370>], [<Reaction HOMt4 at 0x7fc1e05c00a0>], [<Reaction r1891 at 0x7fc2554da430>], [<Reaction HMR_9726 at 0x7fc2435713a0>]]
HOMt4, r1891, HMR_9726, SLGTH

Gapfilling thym with: 
[[<Reaction SLGTH at 0x7fc21b69cd60>], [<Reaction SDPDS_2 at 0x7fc203c00370>], [<Reaction NRRH at 0x7fc203c00490>], [<Reaction SRK at 0x7fc26db617c0>]]
NRRH, SRK, SLGTH, 

In [9]:
# Initialize an empty list to store the extracted lists
gapfill_lists = []

# Open and read the file
with open('gapfill_reactions.txt', 'r') as file:
    lines = file.readlines()

# Process every second line and split it into a list
for i in range(1, len(lines), 3):
    reaction_list = [item.strip() for item in lines[i].strip().split(',')]
    gapfill_lists.extend(reaction_list)

merged_lists = set(gapfill_lists)
merged_lists.discard('')

# Print the extracted lists
print(merged_lists)

{'FACT', 'ARGDC', 'DHPS2_2', 'NRRH', 'CPS_1', 'SUCptspp_1', 'SRK', 'GLY3Pt2', 'SLGTH', 'NP1_2', 'SDPDS_2', 'r1891', 'THDPS_2', 'GLYC3tr', 'HOMt4', 'OCBT2i', 'GLYFEM', 'HMR_9726'}


# 3. Get a short list by testing new reactions
Some reactions in Universal is not well documented and can result in bad model. Therefore, we test the model using database search

In [10]:
short_list = []
for reaction_id in merged_lists:
    # Get the reaction from the reference model
    reaction = universal.reactions.get_by_id(reaction_id)
    model_temp = model.copy()
    
    with model_temp:
        # Add a copy of the reaction to your model
        model_temp.add_reactions([reaction.copy()])
        model_temp.solver ='cplex'
        model_temp.reactions.EX_co2_e.bounds= (0, 1000)

        solution = model_temp.optimize()
        print (f"------\nTest with {reaction_id} \nSolution: {solution}\n--------")

        if solution.objective_value < 1:
            short_list.append(reaction_id)

short_list

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmp88c7cni8.lp
Reading time = 0.02 seconds
: 1886 rows, 4524 columns, 19812 nonzeros
------
Test with FACT 
Solution: <Solution 3.380 at 0x7fc18fed96d0>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmp4232jmv3.lp
Reading time = 0.01 seconds
: 1886 rows, 4524 columns, 19812 nonzeros
------
Test with ARGDC 
Solution: <Solution 0.156 at 0x7fc1703e3490>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpron4fyja.lp
Reading time = 0.01 seconds
: 1886 rows, 4524 columns, 19812 nonzeros
------
Test with DHPS2_2 
Solution: <Solution 0.156 at 0x7fc120ef83d0>
--------
Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmp_c9y5w44.lp
Reading time = 0.02 seconds
: 1886 rows, 4524 columns, 19812 nonzeros
------
Test with NRRH 
Solution: <Solution 0.156 at 0x7fc170943370>
--------
Read LP format model from fi

['ARGDC',
 'DHPS2_2',
 'NRRH',
 'CPS_1',
 'SRK',
 'GLY3Pt2',
 'NP1_2',
 'SDPDS_2',
 'r1891',
 'THDPS_2',
 'GLYC3tr',
 'HOMt4',
 'OCBT2i',
 'HMR_9726']

In [38]:
import requests, re

def get_valid_ec_number(annotations):
    ec_numbers = annotations.get('ec-code', [])
    for ec in ec_numbers:
        # Ensure the EC number has only digits and dots
        if re.match(r'^\d+(\.\d+)+$', ec):
            return ec
    return None

def check_annotation_uniprot(reaction_id, taxonomy_id):
    reaction = universal.reactions.get_by_id(reaction_id)
    ec_number = get_valid_ec_number(reaction.annotation)
    if ec_number is None:
        print(f"No valid EC number found for reaction {reaction_id}")
        return None
    
    print(f"Checking {reaction_id}, with ec {ec_number}")
    query = f'(taxonomy_id:{taxonomy_id}) AND (ec:{ec_number})'
    url = 'https://rest.uniprot.org/uniprotkb/search?'
    params = {
        'query': query,
        'format': 'json',
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve data: {response.text}")
        return None

In [39]:

taxonomy_id = 1886

for r in short_list:
    result = check_annotation_uniprot(r, taxonomy_id)
    print(result)


Checking ARGDC, with ec 4.1.1.19
{"results":[]}
No valid EC number found for reaction DHPS2_2
None
No valid EC number found for reaction NRRH
None
No valid EC number found for reaction CPS_1
None
No valid EC number found for reaction SRK
None
No valid EC number found for reaction GLY3Pt2
None
No valid EC number found for reaction NP1_2
None
No valid EC number found for reaction SDPDS_2
None
No valid EC number found for reaction r1891
None
No valid EC number found for reaction THDPS_2
None
No valid EC number found for reaction GLYC3tr
None
No valid EC number found for reaction HOMt4
None
No valid EC number found for reaction OCBT2i
None
No valid EC number found for reaction HMR_9726
None


In [36]:
# Test added reactions
temp_model = model.copy()
for reaction_id in short_list:
    # Get the reaction from the universal model
    reaction = universal.reactions.get_by_id(reaction_id)
    
    # Add a copy of the reaction to your model
    temp_model.add_reactions([reaction.copy()])

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmp5wr04hgq.lp
Reading time = 0.02 seconds
: 1886 rows, 4524 columns, 19812 nonzeros


In [37]:
def optimize_medium(model, bigg, type, exchange):
    """
    This function does optimize based on a chemical in the certain biolog medium and add an exchange reaction if needed
    """


    # when model has the exchange reaction, no need to add a reaction before gapfilling.    
    with model:
        medium = model.medium
        medium[exchange] = 0.8
        if type.startswith("C"):
            medium["EX_glc__D_e"] = 0

        elif type.startswith("N"):
            medium["EX_nh4_e"] = 0

        elif type.startswith("P"):
            medium["EX_pi_e"] = 0
        
        elif type.startswith("S"):
            medium["EX_so4_e"] = 0

        print(f"\n Optimize {bigg}, {type} with: ")
        model.medium = medium

        model.reactions.EX_co2_e.bounds= (0, 1000)
        model.solver = 'cplex'
        model.objective = 'growth'
        solution = model.optimize()
        print(solution.objective_value)
    return solution 
            
            
for ind in false_negatives.index:
    optimize_medium(temp_model, false_negatives['bigg'][ind], false_negatives['moa'][ind], false_negatives['exchange'][ind])




 Optimize met__L, C-Source, amino acid with: 
0.0

 Optimize citr__L, N-Source, amino acid with: 
0.3253267107541947

 Optimize hom__L, N-Source, amino acid with: 
0.27110559229516656

 Optimize thym, N-Source, other with: 
0.16266335537709795

 Optimize thymd, N-Source, other with: 
0.2982161515246784

 Optimize ura, N-Source, other with: 
0.1693618164538203

 Optimize uri, N-Source, other with: 
0.31049666349867366

 Optimize glyc3p, P-Source, organic with: 
0.24399503306565254


# 3. Gapfill Untestables with Universal model

In [45]:
def add_exchange(model, reference, metabolite_bigg, metabolite_name):
    """
    This function aims to add an extra exchange reaction based on metabolite to the target model using reference model.
    
    Parameters:
    -----------
    metabolite_bigg: String
    metabolite_name: String
    model: Cobra Model
    reference: Cobra Model
    """
    try:
        ex_rxn_id = f"EX_{metabolite_bigg}_e"
        ex_met_id = f"{metabolite_bigg}_e"
        cy_met_id = f"{metabolite_bigg}_c"
        print(f"Try to add {ex_rxn_id} reaction")
        if any(r.id == ex_rxn_id for r in model.reactions.query(ex_rxn_id)):
            if model.reactions.get_by_id(ex_rxn_id).boundary == False:
                model.add_boundary(model.metabolites.get_by_id(ex_met_id), type="exchange")
            return model, ex_rxn_id

        else:
            # Check if the reaction is already in reference model, 
            # if yes, just copy reaction to model,
            # if not, add a new reaction.
            
                if reference.reactions.query(ex_rxn_id) != []:
                    ex_rxn = reference.reactions.get_by_id(ex_rxn_id)
                    model.add_reactions([ex_rxn.copy()])
                    print (f"Exchange: {ex_rxn_id} is added")
                
                else:
                    # Check if the metabolites is in target model in both cytosol and external
                    # if yes, create new reaction based on reference model
                    # if not, create new metabolite
                    if any(m.id == cy_met_id for m in reference.metabolites.query(cy_met_id)):
                        ex_rxn = Reaction(id=ex_rxn_id, name=f"{metabolite_name}-exchange")
                        model.add_reactions([ex_rxn])
                        met = reference.metabolites.get_by_id(cy_met_id).copy()
                        met.id = ex_met_id
                        met.compartment = 'e'
                        ex_rxn.add_metabolites({met: -1})                        
                        print(f"Exchange: {ex_rxn_id} is added")

                    elif any(m.id == ex_met_id for m in reference.metabolites.query(ex_met_id)):
                        ex_rxn = Reaction(id=ex_rxn_id, name=f"{metabolite_name}-exchange")
                        model.add_reactions([ex_rxn])
                        met = reference.metabolites.get_by_id(ex_met_id).copy()
                        ex_rxn.add_metabolites({met: -1})                        
                        print(f"Exchange: {ex_rxn_id} is added")
                    else:
                        print(f'{metabolite_bigg} is not in reference model\n')
                        return model, None
                # Final check to see if the boundary is added
                if model.reactions.get_by_id(ex_rxn_id).boundary == False:
                    model.add_boundary(model.metabolites.get_by_id(met.id), type="exchange")
                return model, ex_rxn_id
    except:
        return model, None

In [44]:
# All Untestable Growth Conditions ()
print(f"\nUntestable conditions:\n")

# Mapping of bigg and biolog model
untestables = pd.read_csv(f"{os.getcwd()}/../untestable_metabolites.csv", converters={'Ignore': lambda x: True if x == 'TRUE' else False})

# Join two tables together
untestables = untestables.merge(agreed_bio_data[['activity', 'index', 'moa', 'bigg']], on='bigg', how='left')
untestables['ignore'] = untestables['ignore'].replace({None: False, 'FALSE': True})
untestables = untestables[["bigg","name", "moa", "ignore"]]
# untestables.to_csv('untestables.csv')
untestables


Untestable conditions:



,bigg,name,moa,ignore
0,2hbut,2 Hydroxybutyrate C4H7O3,"C-Source, carboxylic acid",False
1,2obut,2-Oxobutanoate,"C-Source, carboxylic acid",False
2,2pg,D-Glycerate 2-phosphate,"P-Source, organic",False
3,2pglyc,2-Phosphoglycolate,"P-Source, organic",False
4,35cgmp,"3',5'-Cyclic GMP","P-Source, organic",False
...,...,...,...,...
77,tym,Tyramine,"N-Source, other",False
78,tyrp,Phosphotyrosine,"P-Source, organic",False
79,tyrp,Phosphotyrosine,"P-Source, organic",False
80,urate,Urate C5H4N4O3,"N-Source, other",False


In [46]:
_model = model.copy()
with open('gapfill_reactions_uni_untestable.txt', 'w') as file:
    for ind in untestables.index:
        bigg = untestables['bigg'][ind]
        type = untestables['moa'][ind]
        name = untestables['name'][ind]

        # Only test ones that needs to be added
        if untestables['ignore'][ind] == False:
            file.write(f"Gapfilling {bigg}, {type} with:\n")
            print(f"---------------\nGapfilling {bigg}, {type} test:\n")
            # Try to add exchange reactions before gapfilling
            _model, ex_rxn = add_exchange(_model, universal, bigg, name)
            solution = None
            if ex_rxn:
                solution = gapfill_medium(_model, universal, bigg, type, ex_rxn)

            # Write gapfilling results
            if solution:
                unique_solution = set([reaction for reaction_set in solution for reaction in reaction_set])
                reactions_str = ', '.join(str(reaction.id) for reaction in unique_solution)
                file.write(reactions_str + '\n\n')
            else:
                file.write('\n\n')

Read LP format model from file /var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/tmpr7q_wsns.lp
Reading time = 0.02 seconds
: 1886 rows, 4524 columns, 19812 nonzeros
---------------
Gapfilling 2hbut, C-Source, carboxylic acid test:

Try to add EX_2hbut_e reaction
Exchange: EX_2hbut_e is added

Gapfilling 2hbut with: 
[[<Reaction SLGTH at 0x7fc25dba7640>], [<Reaction GLYFEM at 0x7fc12642aca0>], [<Reaction FACT at 0x7fc124cf5610>], [<Reaction SUCptspp_1 at 0x7fc125f67df0>]]
---------------
Gapfilling 2obut, C-Source, carboxylic acid test:

Try to add EX_2obut_e reaction
Exchange: EX_2obut_e is added

Gapfilling 2obut with: 
[[<Reaction SUCptspp_1 at 0x7fc0a2117880>], [<Reaction SLGTH at 0x7fc1678650d0>], [<Reaction SK_coa_c at 0x7fc1673bb1f0>], [<Reaction 2OBUT_Et at 0x7fc16776e0a0>]]
---------------
Gapfilling 2pg, P-Source, organic test:

Try to add EX_2pg_e reaction
Exchange: EX_2pg_e is added

Gapfilling 2pg with: 
[[<Reaction DHPS2_2 at 0x7fc090a401f0>], [<Reaction CPS_1 at 0x7fc08c9

: 

In [14]:
# Initialize an empty list to store the extracted lists
gapfill_lists = []

# Open and read the file
with open('gapfill_reactions_uni_untestable.txt', 'r') as file:
    lines = file.readlines()

# Process every second line and split it into a list
for i in range(1, len(lines), 3):
    reaction_list = [item.strip() for item in lines[i].strip().split(',')]
    gapfill_lists.extend(reaction_list)

merged_lists = set(gapfill_lists)
merged_lists.discard('')

# Print the extracted lists
print(merged_lists)

{'GAMpts', 'ACGApts', 'ARGAGMt7', 'SALCpts', 'ALLTNt2r'}


In [15]:
for reaction_id in merged_lists:
    # Get the reaction from the universal model
    reaction = universal.reactions.get_by_id(reaction_id)
    
    # Add a copy of the reaction to your model
    model.add_reactions([reaction.copy()])

In [40]:
model.reactions.EX_co2_e.bounds= (0, 1000)

cobra.io.write_sbml_model(temp_model, 'Salb-GEM-Uni-gapfill.xml')